In [3]:
%pip install -U qiskit
%pip install qiskit_optimization

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 5.0/5.0 MB 15.1 MB/s eta 0:00:00
     ---------------------------------------- 9.6/9.6 MB 22.8 MB/s eta 0:00:00
     ------------------------------------- 241.5/241.5 kB 14.5 MB/s eta 0:00:00
     ---------------------------------------- 14.6/14.6 MB 8.8 MB/s eta 0:00:00
     -------------------------------------- 124.7/124.7 kB 7.6 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 29.4 MB/s eta 0:00:00
     ----


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 156.5/156.5 kB 1.6 MB/s eta 0:00:00
     -------------------------------------- 633.5/633.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for docplex: started
  Running setup.py install for docplex: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: docplex is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install amazon-braket-sdk

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 260.3/260.3 kB 2.7 MB/s eta 0:00:00
     -------------------------------------- 115.7/115.7 kB 6.6 MB/s eta 0:00:00
     -------------------------------------- 206.6/206.6 kB 3.1 MB/s eta 0:00:00
     -------------------------------------- 194.8/194.8 kB 3.9 MB/s eta 0:00:00
     -------------------------------------- 135.9/135.9 kB 4.1 MB/s eta 0:00:00
     -------------------------------------- 377.0/377.0 kB 3.4 MB/s eta 0:00:00
     -------------------------------------- 524.0/524.0 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 117.2/117.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -----------------

  DEPRECATION: antlr4-python3-runtime is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install qiskit-optimization

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import boto3
from braket.circuits import Circuit, Gate, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice


import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.algorithms import MinimumEigenOptimizer

from braket.circuits import Circuit
from qiskit.algorithms import QAOA
from qiskit.utils import algorithm_globals


In [ ]:
import numpy as np

from braket.circuits import Circuit

N = 4 # Number of stocks
w = 0.5 # Weighting factor

# Sample values for expected returns, variances, and covariances
r = [0.1, 0.12, 0.15, 0.08] # Expected returns
cov_matrix = np.array([
    [0.1, 0.01, 0.02, 0.03],
    [0.01, 0.15, 0.05, 0.01],
    [0.02, 0.05, 0.25, 0.02],
    [0.03, 0.01, 0.02, 0.2]
]) # Covariance matrix

A = (1-w) * 2 * cov_matrix - w * np.diag(r)

def create_qubo_problem(A):
    N = len(A)
    qp = QuadraticProgram()
    for i in range(N):
        qp.binary_var(f'x{i}')
    qp.minimize(linear=-w*np.array(r), quadratic=A)
    return qp

def get_qubo_from_qp(qp):
    qubo_converter = QuadraticProgramToQubo()
    qubo = qubo_converter.convert(qp)
    return qubo

def qubo_qp_to_matrix(qubo_qp):
    n_qubits = qubo_qp.get_num_vars()
    qubo_matrix = np.zeros((n_qubits, n_qubits))
    
    for (i, j), value in qubo_qp.objective.quadratic.to_dict().items():
        qubo_matrix[i, j] = value
        
    return qubo_matrix

def qubo_to_ising(Q):
    # Create an empty Ising matrix with the same shape as the QUBO matrix
    J = np.zeros(Q.shape)
    
    # Fill the Ising matrix with values from the QUBO matrix
    J = Q / 4
    np.fill_diagonal(J, np.diag(Q) / 2)
    
    # Calculate the offset h
    h = np.sum(Q) / 4
    
    return J, h

n_qubits = 4

qp = create_qubo_problem(A)

qubo_qp = get_qubo_from_qp(qp)

qubo_matrix = qubo_qp_to_matrix(qubo_qp)

J, h = qubo_to_ising(qubo_matrix)

# Set the depth (number of layers) for the QAOA algorithm
p = 2

# Generate random initial values for gammas and betas
initial_gammas = np.random.uniform(0, 2 * np.pi, p)
initial_betas = np.random.uniform(0, np.pi, p)

# Combine gammas and betas into a single array of parameters
initial_params = np.concatenate([initial_gammas, initial_betas])

def driver(beta, n_qubits):
    circ = Circuit()
    for qubit in range(n_qubits):
        gate = Circuit().rx(qubit, 2 * beta)
        circ.add(gate)
    return circ

def cost_circuit(gamma, n_qubits, ising):
    circ = Circuit()
    idx = ising.nonzero()
    edges = list(zip(idx[0], idx[1]))

    for qubit_pair in edges:
        if qubit_pair[0] != qubit_pair[1]:  # Ignore diagonal elements / self-loops
            int_strength = ising[qubit_pair[0], qubit_pair[1]]
            gate = Circuit().zz(qubit_pair[0], qubit_pair[1], angle=2 * gamma * int_strength)
            circ.add(gate)

    return circ

def circuit(params, n_qubits, ising):
    p = len(params) // 2  # number of layers
    gammas, betas = params[:p], params[p:]
    circ = Circuit()

    # Add Hadamard gates to all qubits
    for i in range(n_qubits):
        circ.h(i)
    
    # Apply layers
    for gamma, beta in zip(gammas, betas):
        circ.add(cost_circuit(gamma, n_qubits, ising))
        circ.add(driver(beta, n_qubits))

    return circ

# Create the QAOA circuit using the initial parameters, the number of qubits, and the Ising matrix J
qaoa_circuit = circuit(initial_params, n_qubits, J)


import numpy as np
from scipy.optimize import minimize
from braket.aws import AwsDevice

def evaluate_bitstring(bitstring, qubo):
    energy = 0.0
    for i in range(n_qubits):
        for j in range(n_qubits):
            energy += qubo.objective.quadratic.to_array()[i,j] * int(bitstring[i]) * int(bitstring[j])
        energy += qubo.objective.linear.to_array()[i] * int(bitstring[i])
    return energy

def execute_circuit(params, n_qubits, J, device, shots=1000):
    qaoa_circuit = circuit(params, n_qubits, J)
    task = device.run(qaoa_circuit, shots=shots)
    result = task.result()

    # Calculate the expectation value
    energy = 0
    for item in result.measurement_counts.items():
        bitstring = item[0]
        count = item[1]
        energy += count * evaluate_bitstring(bitstring, qubo_qp)
    
    expectation_value = energy / shots
    return expectation_value


# Define an objective function for the classical optimizer
def objective(params):
    return execute_circuit(params, n_qubits, J, device)


# Choose a quantum device
device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")


# Optimize QAOA parameters using the COBYLA optimizer
result = minimize(objective, initial_params, method="COBYLA")

In [45]:

# Get the optimized parameters
optimized_params = result.x


In [46]:

# Execute the QAOA circuit with the optimized parameters and obtain the optimal solution
qaoa_circuit_optimized = circuit(optimized_params, n_qubits, J)
task_optimized = device.run(qaoa_circuit_optimized, shots=1000)
result_optimized = task_optimized.result()

# Find the bitstring with the highest probability
best_bitstring = max(result_optimized.measurement_counts, key=result_optimized.measurement_counts.get)

print("Optimized parameters:", optimized_params)
print("Optimal solution:", best_bitstring)

Optimized parameters: [ 1.47857341  2.93008479 -0.04783304 -0.29250159]
Optimal solution: 0011
